## 배송 - 전체 파일 함수

In [556]:
# 배송 전체 함수
from konlpy.tag import Okt
import re

def process_delivery_data(file_path):
    # 파일 이름 추출
    file_name = os.path.basename(file_path).split('.')[0]
    
    # 파일 읽기
    df = pd.read_csv(file_path)
    
    # 후처리: '정 사이즈' -> '정사이즈', '온 버핏' -> '오버핏'
    df['Processed_Review'] = df['Processed_Review'].apply(
        lambda x: x.replace('정 사이즈', '정사이즈').replace('온 버핏', '오버핏') if pd.notnull(x) else x
    )
    
    # '버핏' -> '오버핏' 변경 in common_keywords_3_plus
    df['common_keywords_3_plus'] = df['common_keywords_3_plus'].apply(
        lambda x: ', '.join(['오버핏' if keyword.strip() == '버핏' else keyword for keyword in str(x).split(',')]) if pd.notnull(x) else x
    )
    
    # 배송 관련 단어 리스트 (이 외에는 예외 처리)
    shipping_keywords = ["배송", "느리다", "예약배송", "예약", "출고"]
    
    # 'delivery' 여부 분류를 위한 정확한 키워드 매칭 함수
    def is_delivery(keywords):
        keywords = [k.strip() for k in keywords.split(',')]
        
        # 조건 1: shipping_keywords에 정확히 일치하는 키워드가 있는지
        if any(k in shipping_keywords for k in keywords):
            return '배송'
        
        # 조건 2: '깔끔하다'와 정확히 '배송' 또는 '오다'가 함께 있는지
        if '깔끔하다' in keywords and ('배송' in keywords or '오다' in keywords):
            return '배송'
        
        # 조건 3: '빨리'와 '오다'가 함께 있는지
        if '빨리' in keywords and '오다' in keywords:
            return '배송'
        
        # 조건 4: '하루', '이틀', '일주일' 중 하나와 '오다', '온', '만에' 중 하나가 함께 있는지
        if any(k in ['하루', '이틀', '일주일'] for k in keywords) and any(k in ['오다', '온', '만에'] for k in keywords):
            return '배송'
        
        # 조건 5: '빠르다'와 '오다' 또는 '배송'이 함께 있는지
        if '빠르다' in keywords and ('오다' in keywords or '배송' in keywords):
            return '배송'
        
        # 조건 6: '기다리다'와 '오다' 또는 '배송'이 함께 있는지
        if '기다리다' in keywords and ('오다' in keywords or '배송' in keywords):
            return '배송'
        
        return '-'
    
    # '배송' 여부 분류
    df['delivery'] = df['common_keywords_3_plus'].apply(
        lambda x: is_delivery(x) if isinstance(x, str) else '-'
    )
    
    # Okt 형태소 분석기 초기화
    okt = Okt()
    
    # 'Processed_Review_Okt' 열 생성 및 배송 관련 행에 대해 기본형 변환
    df['Processed_Review_Okt'] = df['Processed_Review']
    df.loc[df['delivery'] == '배송', 'Processed_Review_Okt'] = df.loc[df['delivery'] == '배송', 'Processed_Review'].apply(
        lambda x: ' '.join([word[0] for word in okt.pos(x, stem=True)]) if isinstance(x, str) else x
    )
    
    # 쉼표로 연결된 'Processed_Review_Okt'
    df['Processed_Review_Okt'] = df['Processed_Review_Okt'].apply(
        lambda x: ', '.join(x.split()) if isinstance(x, str) else x
    )
    
    # '배송' 관련 행 필터링
    delivery_filtered = df[df['delivery'] == '배송'][['Processed_Review', 'common_keywords_3_plus', 'delivery', 'Processed_Review_Okt']]
    
    # 빠르다/느리다 관련 키워드 리스트
    fast_keywords = ["빨", "로켓", "반나절", "일찍"]
    slow_keywords = ["느리다", "예약배송", "예약", "걸리다", "출고", "늦어지다", "아쉽다", "늦다", "감점", "넘다", "늦", "지연"]
    
    # 'speed' 열 추가하여 빠르다/느리다 분류
    def classify_speed(x):
        if not isinstance(x, str):
            return '만족하다'
        
        # '빠르다' 분류 조건
        if (
            any(keyword in x for keyword in fast_keywords) or
            ('빠르다' in x and ('오다' in x or '배송' in x)) or
            ('오다' in x and ('하루' in x or '빨리' in x)) or
            ('하루' in x and ('온' in x or '만에' in x)) or
            ('깔끔하다' in x and '오다' in x) or
            ('칼' in x and ('오다' in x or '배송' in x))
        ):
            return '빠르다'
        
        # '느리다' 분류 조건
        if (
            any(keyword in x for keyword in slow_keywords) or
            ('이틀' in x and ('온' in x or '만에' in x or '오다' in x)) or
            ('일주일' in x and ('온' in x or '만에' in x or '오다' in x))
        ):
            return '느리다'
        
        return '만족하다'
    
    delivery_filtered['speed'] = delivery_filtered['Processed_Review_Okt'].apply(classify_speed)
    
    # 빠르다/느리다의 개수와 비율 계산
    speed_nums = delivery_filtered['speed'].value_counts()
    total_nums = len(delivery_filtered)
    
    # '느리다'가 없으면 '만족하다'로 대체
    if '느리다' not in speed_nums.index:
        speed_nums['만족하다'] = total_nums - speed_nums.get('빠르다', 0)
    
    # 비율 계산
    speed_ratios = (speed_nums / total_nums) * 100
    speed_ratios_format = speed_ratios.apply(lambda x: f"{x:.2f}%")
    
    # '느리다'가 있는지 확인하여 출력할 결과 설정
    if '느리다' in speed_nums.index:
        # '빠르다'와 '느리다'의 비율만 필터링
        filter_speed_ratio = speed_ratios_format[speed_ratios_format.index.isin(['빠르다', '느리다'])]
    else:
        # '빠르다'와 '만족하다'의 비율만 필터링
        filter_speed_ratio = speed_ratios_format[speed_ratios_format.index.isin(['빠르다', '만족하다'])]
    
    # 결과 출력
    print(f"{file_name}.csv")
    for index, value in filter_speed_ratio.items():
        print(f"{index}    {value}")
    
    return delivery_filtered

# 해시태그병합 폴더 내 모든 CSV 파일에 대해 함수 실행
def process_all_files_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)
            result_df = process_delivery_data(file_path)    # 결과 출력
            print("")

# 함수 실행
folder_path = './해시태그병합'  # 폴더 경로
process_all_files_in_folder(folder_path)

1645762_keywords.csv
빠르다    86.36%
느리다    4.55%

1875663_keywords.csv
빠르다    93.18%
만족하다    6.82%

1875664_keywords.csv
빠르다    85.37%
느리다    7.32%

1944554_keywords.csv
빠르다    94.64%
느리다    1.79%

2551401_keywords.csv
빠르다    80.65%
느리다    3.23%

2978106_keywords.csv
빠르다    88.89%
느리다    3.70%

3295891_keywords.csv
빠르다    62.50%
느리다    37.50%

3977452_keywords.csv
빠르다    66.67%
느리다    33.33%

3977488_keywords.csv
만족하다    100.00%

4018731_keywords.csv
빠르다    81.82%
느리다    18.18%

460G 컷 헤비 피그먼트 티셔츠-차콜-_keywords.csv
빠르다    73.77%
느리다    18.03%

ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이_keywords.csv
빠르다    64.86%
느리다    21.62%

Deep One Tuck Sweat Shorts [Grey]_keywords.csv
빠르다    76.92%
느리다    15.38%

DOODLE HEART HALF T WHITE GREYISH BLUE_keywords.csv
빠르다    87.14%
느리다    5.71%

TAG OG TEE - WHITE_keywords.csv
빠르다    82.79%
느리다    7.38%

[16수 코마사] 2PACK SMALL ARCH T-SHIRT WHITE _ BLACK_keywords.csv
빠르다    87.04%
느리다    7.41%

[2PACK] EL 스트리트 아트워크 오버핏 반팔티 2종 2PACK_keywords.csv
빠르다    87.07%
느리다    6.0

## 색감 - 전체 파일 함수

In [585]:
# 색감 전체 함수
from konlpy.tag import Okt

def process_color_data(file_path):
    # 파일 이름 추출
    file_name = os.path.basename(file_path).split('.')[0]
    
    # CSV 파일 읽기
    df = pd.read_csv(file_path)
    
    # '버핏' -> '오버핏' 변경
    df['common_keywords_3_plus'] = df['common_keywords_3_plus'].apply(
        lambda x: ', '.join(['오버핏' if keyword.strip() == '버핏' else keyword for keyword in str(x).split(',')]) if pd.notnull(x) else x
    )
    
    # 색감 관련 키워드 리스트
    color_keywords = ["색감", "색상", "색", "밝다", "어둡다", "밝은", "어두운", "무채색", "크림색", "흰색", "하얀색", "화이트", "빨간색", 
                      "회색", "검은색", "보랏빛", "네이비", "브라운", "베이지", "보라색", "검정", "블랙", "그레이", "블루", "초록색", "그린", 
                      "진하다", "오묘하다"]

    # '색감' 분류 (정확한 단어 매칭)
    df['color'] = df['common_keywords_3_plus'].apply(
        lambda x: '색감' if isinstance(x, str) and any(keyword.strip() in color_keywords for keyword in x.split(',')) else '-'
    )
    
    # Okt 형태소 분석기 초기화
    okt = Okt()
    
    # 'Processed_Review_Okt' 열을 만들고, 'color'가 '색감'인 행만 Okt 적용
    df['Processed_Review_Okt'] = df['Processed_Review']
    color_filter = df['color'] == '색감'
    df.loc[color_filter, 'Processed_Review_Okt'] = df.loc[color_filter, 'Processed_Review'].apply(
        lambda x: ' '.join(okt.morphs(x, stem=False)) if isinstance(x, str) else x
    )
    
    # 'Processed_Review_Okt'에서 쉼표로 연결
    df['Processed_Review_Okt'] = df['Processed_Review_Okt'].apply(
        lambda x: ', '.join(x.split()) if isinstance(x, str) else x
    )
    
    # '색감'인 행만 필터링
    color_filtered = df[df['color'] == '색감'][['Processed_Review', 'common_keywords_3_plus', 'color', 'Processed_Review_Okt']]
    
    # 밝다/어둡다 관련 키워드 리스트
    light_keywords = ["밝다", "밝은", "크림색", "흰색", "하얀색", "화이트", "빨간색", "베이지"]
    dark_keywords = ["어둡다", "어두운", "회색", "검은색", "보랏빛", "네이비", "브라운", "보라색", "검정", "블랙", "그레이", "블루", "초록색", "그린"]
    
    # brightness 열 추가하여 밝다/어둡다/만족 분류
    color_filtered['brightness'] = color_filtered['Processed_Review_Okt'].apply(
        lambda x: '생각보다 밝아요' if isinstance(x, str) and any(keyword in x for keyword in light_keywords)
        else ('생각보다 어두워요' if isinstance(x, str) and any(keyword in x for keyword in dark_keywords) else '만족해요')
    )
    
    # '생각보다 밝아요', '생각보다 어두워요', '만족해요'의 개수 계산
    color_counts = color_filtered['brightness'].value_counts()
    
    # 전체 행 수
    total_counts = len(color_filtered)
    
    # '생각보다 밝아요', '생각보다 어두워요', '만족해요'의 비율 계산
    color_ratios = (color_counts / total_counts) * 100
    color_ratios_formatted = color_ratios.apply(lambda x: f"{x:.2f}%")
    
    # 인덱스 순서를 '생각보다 밝아요', '만족해요', '생각보다 어두워요'로 맞춤
    sorted_order = ['생각보다 밝아요', '만족해요', '생각보다 어두워요']
    color_ratios_sorted = color_ratios_formatted.reindex(sorted_order).fillna("0%")  # NaN을 0%로 대체

    # 결과 출력
    print(f"{file_name}.csv")
    for index, value in color_ratios_sorted.items():
        print(f"{index}    {value}")
    
    # 최종 필터된 DataFrame 반환
    return color_filtered

# 해시태그병합 폴더 내 모든 CSV 파일에 대해 함수 실행
def process_all_files_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)
            result_df = process_color_data(file_path)
            print("")  # 파일 사이에 빈 줄 추가

# 함수 실행
folder_path = './해시태그병합'  # 폴더 경로
process_all_files_in_folder(folder_path)

1645762_keywords.csv
생각보다 밝아요    57.24%
만족해요    38.82%
생각보다 어두워요    3.95%

1875663_keywords.csv
생각보다 밝아요    13.21%
만족해요    24.53%
생각보다 어두워요    62.26%

1875664_keywords.csv
생각보다 밝아요    15.07%
만족해요    28.77%
생각보다 어두워요    56.16%

1944554_keywords.csv
생각보다 밝아요    0%
만족해요    67.96%
생각보다 어두워요    32.04%

2551401_keywords.csv
생각보다 밝아요    17.95%
만족해요    28.21%
생각보다 어두워요    53.85%

2978106_keywords.csv
생각보다 밝아요    44.12%
만족해요    42.16%
생각보다 어두워요    13.73%

3295891_keywords.csv
생각보다 밝아요    26.92%
만족해요    53.85%
생각보다 어두워요    19.23%

3977452_keywords.csv
생각보다 밝아요    37.50%
만족해요    18.75%
생각보다 어두워요    43.75%

3977488_keywords.csv
생각보다 밝아요    60.87%
만족해요    8.70%
생각보다 어두워요    30.43%

4018731_keywords.csv
생각보다 밝아요    65.00%
만족해요    25.00%
생각보다 어두워요    10.00%

460G 컷 헤비 피그먼트 티셔츠-차콜-_keywords.csv
생각보다 밝아요    1.48%
만족해요    92.25%
생각보다 어두워요    6.27%

ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이_keywords.csv
생각보다 밝아요    0.85%
만족해요    81.70%
생각보다 어두워요    17.45%

Deep One Tuck Sweat Shorts [Grey]_keywords.csv
생각보다 밝아요    0%

## 배송 (1) - Okt와 기본형 둘 다 적용 x

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import re
import csv

In [96]:
file_path = './해시태그병합/DOODLE HEART HALF T WHITE GREYISH BLUE_keywords.csv'

# 파일 이름 추출
file_name = file_path.split('/')[-1].split('.')[0]   # 마지막 슬래시(/)와 .csv를 기준으로 파일 이름 추출

# 파일 읽기
df = pd.read_csv(file_path)
df

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus
0,"그래픽도 예쁜데, 이 제품은 핏이 정말 너무 좋습니다. 너무 길지 않은 스트릿한 티...",그래픽도 예쁜데 이 제품은 핏이 정말 너무 좋습니다 너무 길지 않은 스트리트 한 티...,그래픽,"그래픽, 길다, 딱이다, 스트리트, 제품, 찾다, 티셔츠, 핏"
1,디자인이 너무 예쁘고 그렇게 두껍지도 않아서 한여름에도 입을만 해요,디자인이 너무 예쁘고 그렇게 두껍지도 않아서 한여름에도 입을만해요,NaN,"그렇게, 두껍다, 디자인, 만해, 한여름"
2,사이즈도 잘맞고 디자인이랑 프린팅 다 너무 마음에 들어요!,사이즈도 잘 맞고 디자인이랑 프린팅 다 너무 마음에 들어요,프린팅,"디자인, 맞다, 사이즈, 프린팅"
3,넘 예뻐요! 프린팅이 안 유치해서 마음에 들어요! 잘 입을게요,너무 예뻐요 프린팅이 안 유치해서 마음에 들어요 잘 입을게요,NaN,"유치하다, 프린팅"
4,커플티로 구매 했는데 너무 이쁘고 흔하지 않아서 더 좋아요 색감도 잘빠지고 프린팅도...,커플티로 구매했는데 너무 이쁘고 흔하지 않아서 더 좋아요 색감도 잘빠지고 프린팅도 ...,프린팅,"벗겨지다, 색감, 잘빠지다, 커플티, 프린팅, 흔하다"
...,...,...,...,...
874,커플티로 샀어요 로고 그래픽 이쁜데 빨면 연해지니까 찬물세탁하세요~,커플티로 샀어요 로고 그래픽 이쁜데 빨면 연해지니까 찬물 세탁하세요,그래픽,"그래픽, 로고, 세탁, 찬물, 커플티, 해지"
875,요즘 유행하길래 사봤는데 너무좋네요 재질도 나름 탄탄하구요 앞에 프린팅이 멋있어서 ...,요즘 유행하길래 사봤는데 너무 좋네요 재질도 나름 탄탄하고요 앞에 프린팅이 멋있어서...,재질,"나름, 사보다, 어울리다, 유행, 재질, 탄탄하다, 프린팅"
876,저번에 티셔츠 산 이후로 원단도 좋고 핏도 좋아서 또 샀어요!,저번에 티셔츠 산 이후로 원단도 좋고 핏도 좋아서 또 샀어요,NaN,"원단, 이후, 저번, 티셔츠, 핏"
877,핏도 예쁘고 디자인도 예쁩니다 손이많이 가요 많이 파세여,핏도 예쁘고 디자인도 예쁩니다 손이 많이 가요 많이 파세요,NaN,"가요, 디자인, 손, 파다, 핏"


In [97]:
# common_keywords_3_plus 키워드 버핏 -> 오버핏으로
df['common_keywords_3_plus'] = df['common_keywords_3_plus'].apply(lambda x: ', '.join(['오버핏' if keyword.strip() == '버핏' else keyword for keyword in str(x).split(',')]) if pd.notnull(x) else x)
df

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus
0,"그래픽도 예쁜데, 이 제품은 핏이 정말 너무 좋습니다. 너무 길지 않은 스트릿한 티...",그래픽도 예쁜데 이 제품은 핏이 정말 너무 좋습니다 너무 길지 않은 스트리트 한 티...,그래픽,"그래픽, 길다, 딱이다, 스트리트, 제품, 찾다, 티셔츠, 핏"
1,디자인이 너무 예쁘고 그렇게 두껍지도 않아서 한여름에도 입을만 해요,디자인이 너무 예쁘고 그렇게 두껍지도 않아서 한여름에도 입을만해요,NaN,"그렇게, 두껍다, 디자인, 만해, 한여름"
2,사이즈도 잘맞고 디자인이랑 프린팅 다 너무 마음에 들어요!,사이즈도 잘 맞고 디자인이랑 프린팅 다 너무 마음에 들어요,프린팅,"디자인, 맞다, 사이즈, 프린팅"
3,넘 예뻐요! 프린팅이 안 유치해서 마음에 들어요! 잘 입을게요,너무 예뻐요 프린팅이 안 유치해서 마음에 들어요 잘 입을게요,NaN,"유치하다, 프린팅"
4,커플티로 구매 했는데 너무 이쁘고 흔하지 않아서 더 좋아요 색감도 잘빠지고 프린팅도...,커플티로 구매했는데 너무 이쁘고 흔하지 않아서 더 좋아요 색감도 잘빠지고 프린팅도 ...,프린팅,"벗겨지다, 색감, 잘빠지다, 커플티, 프린팅, 흔하다"
...,...,...,...,...
874,커플티로 샀어요 로고 그래픽 이쁜데 빨면 연해지니까 찬물세탁하세요~,커플티로 샀어요 로고 그래픽 이쁜데 빨면 연해지니까 찬물 세탁하세요,그래픽,"그래픽, 로고, 세탁, 찬물, 커플티, 해지"
875,요즘 유행하길래 사봤는데 너무좋네요 재질도 나름 탄탄하구요 앞에 프린팅이 멋있어서 ...,요즘 유행하길래 사봤는데 너무 좋네요 재질도 나름 탄탄하고요 앞에 프린팅이 멋있어서...,재질,"나름, 사보다, 어울리다, 유행, 재질, 탄탄하다, 프린팅"
876,저번에 티셔츠 산 이후로 원단도 좋고 핏도 좋아서 또 샀어요!,저번에 티셔츠 산 이후로 원단도 좋고 핏도 좋아서 또 샀어요,NaN,"원단, 이후, 저번, 티셔츠, 핏"
877,핏도 예쁘고 디자인도 예쁩니다 손이많이 가요 많이 파세여,핏도 예쁘고 디자인도 예쁩니다 손이 많이 가요 많이 파세요,NaN,"가요, 디자인, 손, 파다, 핏"


In [98]:
# 포함 단어 리스트
shipping_keywords = ["배송", "빠르다", "느리다", "예약배송", "깔끔하다", "예약", "오다", "하루", 
                     "이틀", "기다리다", "빨리", "일주일", "포장", "출고"]

# 새로운 열 '배송'을 추가하여 단어가 포함된 경우 '배송'으로 분류
df['delivery'] = df['common_keywords_3_plus'].apply(
    lambda x: '배송' if isinstance(x, str) and any(keyword in x for keyword in shipping_keywords) else '-'
)

df

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus,delivery
0,"그래픽도 예쁜데, 이 제품은 핏이 정말 너무 좋습니다. 너무 길지 않은 스트릿한 티...",그래픽도 예쁜데 이 제품은 핏이 정말 너무 좋습니다 너무 길지 않은 스트리트 한 티...,그래픽,"그래픽, 길다, 딱이다, 스트리트, 제품, 찾다, 티셔츠, 핏",-
1,디자인이 너무 예쁘고 그렇게 두껍지도 않아서 한여름에도 입을만 해요,디자인이 너무 예쁘고 그렇게 두껍지도 않아서 한여름에도 입을만해요,NaN,"그렇게, 두껍다, 디자인, 만해, 한여름",-
2,사이즈도 잘맞고 디자인이랑 프린팅 다 너무 마음에 들어요!,사이즈도 잘 맞고 디자인이랑 프린팅 다 너무 마음에 들어요,프린팅,"디자인, 맞다, 사이즈, 프린팅",-
3,넘 예뻐요! 프린팅이 안 유치해서 마음에 들어요! 잘 입을게요,너무 예뻐요 프린팅이 안 유치해서 마음에 들어요 잘 입을게요,NaN,"유치하다, 프린팅",-
4,커플티로 구매 했는데 너무 이쁘고 흔하지 않아서 더 좋아요 색감도 잘빠지고 프린팅도...,커플티로 구매했는데 너무 이쁘고 흔하지 않아서 더 좋아요 색감도 잘빠지고 프린팅도 ...,프린팅,"벗겨지다, 색감, 잘빠지다, 커플티, 프린팅, 흔하다",-
...,...,...,...,...,...
874,커플티로 샀어요 로고 그래픽 이쁜데 빨면 연해지니까 찬물세탁하세요~,커플티로 샀어요 로고 그래픽 이쁜데 빨면 연해지니까 찬물 세탁하세요,그래픽,"그래픽, 로고, 세탁, 찬물, 커플티, 해지",-
875,요즘 유행하길래 사봤는데 너무좋네요 재질도 나름 탄탄하구요 앞에 프린팅이 멋있어서 ...,요즘 유행하길래 사봤는데 너무 좋네요 재질도 나름 탄탄하고요 앞에 프린팅이 멋있어서...,재질,"나름, 사보다, 어울리다, 유행, 재질, 탄탄하다, 프린팅",-
876,저번에 티셔츠 산 이후로 원단도 좋고 핏도 좋아서 또 샀어요!,저번에 티셔츠 산 이후로 원단도 좋고 핏도 좋아서 또 샀어요,NaN,"원단, 이후, 저번, 티셔츠, 핏",-
877,핏도 예쁘고 디자인도 예쁩니다 손이많이 가요 많이 파세여,핏도 예쁘고 디자인도 예쁩니다 손이 많이 가요 많이 파세요,NaN,"가요, 디자인, 손, 파다, 핏",-


In [99]:
# 'delivery' 열에서 '배송'으로 분류된 행만 필터링
delivery_filtered = df[df['delivery'] == '배송']
delivery_filtered

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus,delivery
16,"기다린 보람이 있는 녀석,,, 내 마음을 저격 성공,,, 넘옙흐네여",기다린 보람이 있는 녀석 내 마음을 저격 성공 넘옙흐네여,기다리다,"기다리다, 넘옙흐네, 녀석, 보람, 성공, 저격",배송
25,사이즈 살짝 크게 나온것 같아용 스몰 샀는데 괜찮네요! 이쁩니다 올여름 잘입겠슴니다,사이즈 살짝 크게 나온 것 같아요 스몰 샀는데 괜찮네요 이쁩니다 올여름 잘 입겠습니다,NaN,"나오다, 사이즈, 스몰, 올여름, 크게",배송
36,일단 옷이 너무 이쁘고 디테일이 너무 좋아요 포장부터 메종미네드 느낌 팍팍나게 아주...,일단 옷이 너무 이쁘고 디테일이 너무 좋아요 포장부터 메종미네드 느낌 팍팍 나게 아...,NaN,"나다, 나일론, 바지, 세련되다, 찰떡, 청바지, 포장",배송
39,배송도 빠르고 넥라인 쫀쫀해서 너무 좋네요 데일리로 입기 땃 좋습니다,배송도 빠르고 넥 라인 쫀쫀해서 너무 좋네요 데일리로 입기 뜻 좋습니다,NaN,"데일리, 라인, 배송, 빠르다, 쫀쫀해",배송
44,기다리고 기다린 예약배송상품.. 기다린 보람이 충분히 있을정도로 예뻤습니다 너무만족해요,기다리고 기다린 예약배송 상품 기다린 보람이 충분히 있을 정도로 예뻤습니다 너무 만족해요,충분하다,"기다리다, 만족하다, 배송, 보람, 상품, 예약, 충분하다",배송
...,...,...,...,...,...
830,색감화면이랑비슷 배송빠르고좋아요 가격도합리적이고 좋아요!!!!!!!,색감 화면이랑 비슷 배송 빠르고 좋아요 가격도 합리적이고 좋아요,NaN,"가격, 배송, 비슷, 빠르다, 색감, 이고, 합리, 화면",배송
831,너무 좋아요 다들 추천합니다 배송도 빠르고 좋아요,너무 좋아요 다들 추천합니다 배송도 빠르고 좋아요,NaN,"배송, 빠르다, 추천",배송
856,배송도 빠르고 디자인도 사진보다 실물이 훨씬 이뿌네용,배송도 빠르고 디자인도 사진보다 실물이 훨씬 이쁘네요,NaN,"디자인, 배송, 빠르다, 사진, 실물",배송
859,배송은 하루만에 왔어요. 가격은 조금 비싼듯 합니다.,배송은 하루 만에 왔어요 가격은 조금 비싼듯합니다,"가격, 하루","가격, 배송, 비싸다, 오다, 하루",배송


In [100]:
# 빠르다와 느리다 관련 단어 리스트
fast_keywords = ["빠르다", "깔끔하다", "하루", "빨리", "오다"]
slow_keywords = ["느리다", "이틀", "기다리다", "일주일", "예약배송", "예약"]

# 원본 DataFrame을 수정하거나 슬라이스된 데이터프레임을 복사하여 작업
delivery_related_rows = delivery_filtered.copy()

# .loc를 사용하여 speed 열 추가
delivery_related_rows.loc[:, 'speed'] = delivery_related_rows['common_keywords_3_plus'].apply(
    lambda x: '빠르다' if isinstance(x, str) and any(keyword in x for keyword in fast_keywords)
              else ('느리다' if isinstance(x, str) and any(keyword in x for keyword in slow_keywords) else '-')
)

delivery_related_rows

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus,delivery,speed
16,"기다린 보람이 있는 녀석,,, 내 마음을 저격 성공,,, 넘옙흐네여",기다린 보람이 있는 녀석 내 마음을 저격 성공 넘옙흐네여,기다리다,"기다리다, 넘옙흐네, 녀석, 보람, 성공, 저격",배송,느리다
25,사이즈 살짝 크게 나온것 같아용 스몰 샀는데 괜찮네요! 이쁩니다 올여름 잘입겠슴니다,사이즈 살짝 크게 나온 것 같아요 스몰 샀는데 괜찮네요 이쁩니다 올여름 잘 입겠습니다,NaN,"나오다, 사이즈, 스몰, 올여름, 크게",배송,빠르다
36,일단 옷이 너무 이쁘고 디테일이 너무 좋아요 포장부터 메종미네드 느낌 팍팍나게 아주...,일단 옷이 너무 이쁘고 디테일이 너무 좋아요 포장부터 메종미네드 느낌 팍팍 나게 아...,NaN,"나다, 나일론, 바지, 세련되다, 찰떡, 청바지, 포장",배송,-
39,배송도 빠르고 넥라인 쫀쫀해서 너무 좋네요 데일리로 입기 땃 좋습니다,배송도 빠르고 넥 라인 쫀쫀해서 너무 좋네요 데일리로 입기 뜻 좋습니다,NaN,"데일리, 라인, 배송, 빠르다, 쫀쫀해",배송,빠르다
44,기다리고 기다린 예약배송상품.. 기다린 보람이 충분히 있을정도로 예뻤습니다 너무만족해요,기다리고 기다린 예약배송 상품 기다린 보람이 충분히 있을 정도로 예뻤습니다 너무 만족해요,충분하다,"기다리다, 만족하다, 배송, 보람, 상품, 예약, 충분하다",배송,느리다
...,...,...,...,...,...,...
830,색감화면이랑비슷 배송빠르고좋아요 가격도합리적이고 좋아요!!!!!!!,색감 화면이랑 비슷 배송 빠르고 좋아요 가격도 합리적이고 좋아요,NaN,"가격, 배송, 비슷, 빠르다, 색감, 이고, 합리, 화면",배송,빠르다
831,너무 좋아요 다들 추천합니다 배송도 빠르고 좋아요,너무 좋아요 다들 추천합니다 배송도 빠르고 좋아요,NaN,"배송, 빠르다, 추천",배송,빠르다
856,배송도 빠르고 디자인도 사진보다 실물이 훨씬 이뿌네용,배송도 빠르고 디자인도 사진보다 실물이 훨씬 이쁘네요,NaN,"디자인, 배송, 빠르다, 사진, 실물",배송,빠르다
859,배송은 하루만에 왔어요. 가격은 조금 비싼듯 합니다.,배송은 하루 만에 왔어요 가격은 조금 비싼듯합니다,"가격, 하루","가격, 배송, 비싸다, 오다, 하루",배송,빠르다


In [101]:
# '빠르다'와 '느리다'의 개수 계산
speed_counts = delivery_related_rows['speed'].value_counts()

# 전체 행 수
total_count = len(delivery_related_rows)

# 빠르다와 느리다의 비율 계산
speed_ratio = (speed_counts / total_count) * 100

# 결과를 소수점 둘째 자리까지 포맷팅하고 % 추가
speed_ratio_formatted = speed_ratio.apply(lambda x: f"{x:.2f}%")

# '빠르다'와 '느리다'의 비율만 필터링 (기타 "-"는 제외)
filtered_speed_ratio = speed_ratio_formatted[speed_ratio_formatted.index.isin(['빠르다', '느리다'])]

# 결과 출력
print(f"배송 비율 (배송 관련 행 수 {total_count}개 대비):")
print(filtered_speed_ratio)

배송 비율 (배송 관련 행 수 120개 대비):
speed
빠르다    76.67%
느리다     5.00%
Name: count, dtype: object


In [102]:
print("빠르다와 느리다의 행 수:")
print(speed_counts)

빠르다와 느리다의 행 수:
speed
빠르다    92
-      22
느리다     6
Name: count, dtype: int64


In [103]:
# 위에 비율: 빠르다,느리다 행 수 / 배송 관련 행 수
# 밑에 비율: 빠르다,느리다 행 수 / csv 파일 전체 행 수

# CSV 파일 전체 행 수 계산
total_rows = len(df)

# '빠르다'와 '느리다'의 개수 계산
speed_counts = delivery_related_rows['speed'].value_counts()

# 빠르다, 느리다 비율을 CSV 전체 행 수로 계산
speed_ratio_overall = (speed_counts / total_rows) * 100

# 결과를 소수점 둘째 자리까지 포맷팅하고 % 추가
speed_ratio_overall_formatted = speed_ratio_overall.apply(lambda x: f"{x:.2f}%")

# '빠르다'와 '느리다'의 비율만 필터링 (기타 "-"는 제외)
filtered_speed_overall_ratio = speed_ratio_overall_formatted[speed_ratio_overall_formatted.index.isin(['빠르다', '느리다'])]

# 결과 출력
print(f"배송 비율 (전체 행 수 {total_rows}개 대비):")
print(filtered_speed_overall_ratio)

배송 비율 (전체 행 수 879개 대비):
speed
빠르다    10.47%
느리다     0.68%
Name: count, dtype: object


## 배송 (2) - Okt 적용 -> 기본형 변환

In [568]:
file_path = './해시태그병합/3977488_keywords.csv'

# 파일 이름 추출
file_name = file_path.split('/')[-1].split('.')[0]   # 마지막 슬래시(/)와 .csv를 기준으로 파일 이름 추출

# 파일 읽기
df = pd.read_csv(file_path)
df

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus
0,글로니 말모말모🖤 글로니 핏 이쁜거 두말하면 입아파요\n\n168/51\n허리 얇고...,글로리 말로 말 모 글로리 핏 이쁜 거 두말하면 입 아파요 허리가 늘고 긴 체형 어...,NaN,"가슴, 개, 덜하다, 듬직하다, 숨기다, 어깨, 여전하다, 치마, 태평양, 허리"
1,예약배송이라 쇼룸에서 사이즈 입어보고 샀는데 좀.. 다른 거 같아요 배송된게 좀 더...,예약배송이라 쇼룸에서 사이즈 입어보고 샀는데 좀 다른 거 같아요 배송된 게 좀 더 ...,NaN,"대신, 민소매, 배송, 부담스럽다, 불편하다, 사이즈, 소재, 쇼룸, 수영복, 입어..."
2,화사해보이고 예쁘지만 기장이 조금만 더 길었느면 좋겠음\n많이 짧아서 좀 부담스러울수도,화사해 보이고 예쁘지만 기장이 조금만 더 길었으면 좋겠음 많이 짧아서 좀 부담스러울 수도,"기장, 짧다, 화사하다","기장, 길다, 보이, 부담스럽다, 수도, 짧다, 화사하다"
3,두겹이라 비치지도 않고 \n완전 탄탄합니다\n올해 원피스중 제일 이쁜듯요\n너무 맘...,두 겹이라 비치지도 않고 정말 탄탄합니다 올해 원피스 중 제일 이쁜 듯요 너무 맘에...,NaN,"겹, 들어서다, 비치다, 올해, 원피스, 칼라, 탄탄하다"
4,재질이 탄탄해서 흰색임에도 비침이 거의 없어서 좋았어요. 노란기나 푸른기 없이 깨끗...,재질이 탄탄해서 흰색임에도 비침이 거의 없어서 좋았어요 노란 기나 푸른 기 없이 깨...,NaN,"깨끗하다, 노랗다, 사이즈, 색깔, 신경, 재질, 정석, 탄탄하다, 흰색"
...,...,...,...,...
88,작은 제 키에는 살짝 긴 감이 있지만 무척 예쁘고 편합니다.,작은 제 키에는 살짝 긴 감이 있지만 무척 예쁘고 편합니다,NaN,"무척, 에는, 작다, 키, 편하다"
89,딱 붙는 핏 찾았었는데 생각한대로에요 여기저기 입기 좋아요,딱 붙는 핏 찾았었는데 생각한 대로에요 여기저기 입기 좋아요,대로,"대로, 붙다, 여기저기, 찾다"
90,믿고 사는 글로니 핏 너무 예뻐요 완전 깔끔해요 다른색도 살거에요,믿고 사는 글로리 핏 너무 예뻐요 정말 깔끔해요 다른 색도 살 거예요,살다,"글로리, 깔끔하다, 믿다, 살다, 색도"
91,블랙도 사고 싶을 정도류 예쁜데 너무짧음 소재는 너무 얇지 않아 좋아요,블랙도 사고 싶을 정도류 예쁜데 너무 짧음 소재는 너무 얇지 않아 좋아요,"짧다, 소재","류, 블랙, 사고, 소재, 얇다, 짧다"


In [569]:
# 후처리
# '정 사이즈' -> '정사이즈'
df['Processed_Review'] = df['Processed_Review'].apply(
    lambda x: x.replace('정 사이즈', '정사이즈') if pd.notnull(x) else x)

# '온 버핏' -> '오버핏'
df['Processed_Review'] = df['Processed_Review'].apply(
    lambda x: x.replace('온 버핏', '오버핏') if pd.notnull(x) else x)

# '버핏' -> '오버핏' 변경
df['common_keywords_3_plus'] = df['common_keywords_3_plus'].apply(
    lambda x: ', '.join(['오버핏' if keyword.strip() == '버핏' else keyword for keyword in str(x).split(',')]) if pd.notnull(x) else x
)
df

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus
0,글로니 말모말모🖤 글로니 핏 이쁜거 두말하면 입아파요\n\n168/51\n허리 얇고...,글로리 말로 말 모 글로리 핏 이쁜 거 두말하면 입 아파요 허리가 늘고 긴 체형 어...,NaN,"가슴, 개, 덜하다, 듬직하다, 숨기다, 어깨, 여전하다, 치마, 태..."
1,예약배송이라 쇼룸에서 사이즈 입어보고 샀는데 좀.. 다른 거 같아요 배송된게 좀 더...,예약배송이라 쇼룸에서 사이즈 입어보고 샀는데 좀 다른 거 같아요 배송된 게 좀 더 ...,NaN,"대신, 민소매, 배송, 부담스럽다, 불편하다, 사이즈, 소재, 쇼룸, ..."
2,화사해보이고 예쁘지만 기장이 조금만 더 길었느면 좋겠음\n많이 짧아서 좀 부담스러울수도,화사해 보이고 예쁘지만 기장이 조금만 더 길었으면 좋겠음 많이 짧아서 좀 부담스러울 수도,"기장, 짧다, 화사하다","기장, 길다, 보이, 부담스럽다, 수도, 짧다, 화사하다"
3,두겹이라 비치지도 않고 \n완전 탄탄합니다\n올해 원피스중 제일 이쁜듯요\n너무 맘...,두 겹이라 비치지도 않고 정말 탄탄합니다 올해 원피스 중 제일 이쁜 듯요 너무 맘에...,NaN,"겹, 들어서다, 비치다, 올해, 원피스, 칼라, 탄탄하다"
4,재질이 탄탄해서 흰색임에도 비침이 거의 없어서 좋았어요. 노란기나 푸른기 없이 깨끗...,재질이 탄탄해서 흰색임에도 비침이 거의 없어서 좋았어요 노란 기나 푸른 기 없이 깨...,NaN,"깨끗하다, 노랗다, 사이즈, 색깔, 신경, 재질, 정석, 탄탄하다, 흰색"
...,...,...,...,...
88,작은 제 키에는 살짝 긴 감이 있지만 무척 예쁘고 편합니다.,작은 제 키에는 살짝 긴 감이 있지만 무척 예쁘고 편합니다,NaN,"무척, 에는, 작다, 키, 편하다"
89,딱 붙는 핏 찾았었는데 생각한대로에요 여기저기 입기 좋아요,딱 붙는 핏 찾았었는데 생각한 대로에요 여기저기 입기 좋아요,대로,"대로, 붙다, 여기저기, 찾다"
90,믿고 사는 글로니 핏 너무 예뻐요 완전 깔끔해요 다른색도 살거에요,믿고 사는 글로리 핏 너무 예뻐요 정말 깔끔해요 다른 색도 살 거예요,살다,"글로리, 깔끔하다, 믿다, 살다, 색도"
91,블랙도 사고 싶을 정도류 예쁜데 너무짧음 소재는 너무 얇지 않아 좋아요,블랙도 사고 싶을 정도류 예쁜데 너무 짧음 소재는 너무 얇지 않아 좋아요,"짧다, 소재","류, 블랙, 사고, 소재, 얇다, 짧다"


In [570]:
# 배송 관련 단어 리스트 (이 외에는 예외 처리)
shipping_keywords = ["배송", "느리다", "예약배송", "예약", "출고"]  # 하루, 이틀, 일주일, 빨리, 깔끔하다, 오다, 기다리다 제외

# 'delivery' 여부 분류를 위한 정확한 키워드 매칭 함수
def is_delivery(keywords):
    keywords = [k.strip() for k in keywords.split(',')]
    
    # 조건 1: shipping_keywords에 정확히 일치하는 키워드가 있는지
    if any(k in shipping_keywords for k in keywords):
        return '배송'
    
    # 조건 2: '깔끔하다'와 정확히 '배송' 또는 '오다'가 함께 있는지
    if '깔끔하다' in keywords and ('배송' in keywords or '오다' in keywords):
        return '배송'
    
    # 조건 3: '빨리'와 '오다'가 함께 있는지
    if '빨리' in keywords and '오다' in keywords:
        return '배송'
    
    # 조건 4: '하루', '이틀', '일주일' 중 하나와 '오다', '온', '만에' 중 하나가 함께 있는지
    if any(k in ['하루', '이틀', '일주일'] for k in keywords) and any(k in ['오다', '온', '만에'] for k in keywords):
        return '배송'
    
    # 조건 5: '빠르다'와 '오다' 또는 '배송'이 함께 있는지
    if '빠르다' in keywords and ('오다' in keywords or '배송' in keywords):
        return '배송'
    
    # 조건 6: '기다리다'와 '오다' 또는 '배송'이 함께 있는지
    if '기다리다' in keywords and ('오다' in keywords or '배송' in keywords):
        return '배송'
    
    return '-'

# '배송' 여부 분류
df['delivery'] = df['common_keywords_3_plus'].apply(
    lambda x: is_delivery(x) if isinstance(x, str) else '-'
)

df

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus,delivery
0,글로니 말모말모🖤 글로니 핏 이쁜거 두말하면 입아파요\n\n168/51\n허리 얇고...,글로리 말로 말 모 글로리 핏 이쁜 거 두말하면 입 아파요 허리가 늘고 긴 체형 어...,NaN,"가슴, 개, 덜하다, 듬직하다, 숨기다, 어깨, 여전하다, 치마, 태...",-
1,예약배송이라 쇼룸에서 사이즈 입어보고 샀는데 좀.. 다른 거 같아요 배송된게 좀 더...,예약배송이라 쇼룸에서 사이즈 입어보고 샀는데 좀 다른 거 같아요 배송된 게 좀 더 ...,NaN,"대신, 민소매, 배송, 부담스럽다, 불편하다, 사이즈, 소재, 쇼룸, ...",배송
2,화사해보이고 예쁘지만 기장이 조금만 더 길었느면 좋겠음\n많이 짧아서 좀 부담스러울수도,화사해 보이고 예쁘지만 기장이 조금만 더 길었으면 좋겠음 많이 짧아서 좀 부담스러울 수도,"기장, 짧다, 화사하다","기장, 길다, 보이, 부담스럽다, 수도, 짧다, 화사하다",-
3,두겹이라 비치지도 않고 \n완전 탄탄합니다\n올해 원피스중 제일 이쁜듯요\n너무 맘...,두 겹이라 비치지도 않고 정말 탄탄합니다 올해 원피스 중 제일 이쁜 듯요 너무 맘에...,NaN,"겹, 들어서다, 비치다, 올해, 원피스, 칼라, 탄탄하다",-
4,재질이 탄탄해서 흰색임에도 비침이 거의 없어서 좋았어요. 노란기나 푸른기 없이 깨끗...,재질이 탄탄해서 흰색임에도 비침이 거의 없어서 좋았어요 노란 기나 푸른 기 없이 깨...,NaN,"깨끗하다, 노랗다, 사이즈, 색깔, 신경, 재질, 정석, 탄탄하다, 흰색",-
...,...,...,...,...,...
88,작은 제 키에는 살짝 긴 감이 있지만 무척 예쁘고 편합니다.,작은 제 키에는 살짝 긴 감이 있지만 무척 예쁘고 편합니다,NaN,"무척, 에는, 작다, 키, 편하다",-
89,딱 붙는 핏 찾았었는데 생각한대로에요 여기저기 입기 좋아요,딱 붙는 핏 찾았었는데 생각한 대로에요 여기저기 입기 좋아요,대로,"대로, 붙다, 여기저기, 찾다",-
90,믿고 사는 글로니 핏 너무 예뻐요 완전 깔끔해요 다른색도 살거에요,믿고 사는 글로리 핏 너무 예뻐요 정말 깔끔해요 다른 색도 살 거예요,살다,"글로리, 깔끔하다, 믿다, 살다, 색도",-
91,블랙도 사고 싶을 정도류 예쁜데 너무짧음 소재는 너무 얇지 않아 좋아요,블랙도 사고 싶을 정도류 예쁜데 너무 짧음 소재는 너무 얇지 않아 좋아요,"짧다, 소재","류, 블랙, 사고, 소재, 얇다, 짧다",-


In [571]:
# 'delivery'열에 '배송'인 행 있으면, 'Processed_Review'열에서 기본형으로 변환
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

# 'delivery'가 '배송'인 행의 'Processed_Review' 리뷰를 기본형으로 변환 -> 새로운 열 'Processed_Review_Okt'로 추가
df['Processed_Review_Okt'] = df['Processed_Review']  # 기본적으로 동일한 값을 복사해둠

# '배송'인 행에 대해서만 기본형으로 변환
df.loc[df['delivery'] == '배송', 'Processed_Review_Okt'] = df.loc[df['delivery'] == '배송', 'Processed_Review'].apply(
    lambda x: ' '.join([word[0] for word in okt.pos(x, stem=True)]) if isinstance(x, str) else x
)

# 'Processed_Review_Okt' 열에서 값을 쉼표로 연결
df['Processed_Review_Okt'] = df['Processed_Review_Okt'].apply(
    lambda x: ', '.join(x.split()) if isinstance(x, str) else x
)

df

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus,delivery,Processed_Review_Okt
0,글로니 말모말모🖤 글로니 핏 이쁜거 두말하면 입아파요\n\n168/51\n허리 얇고...,글로리 말로 말 모 글로리 핏 이쁜 거 두말하면 입 아파요 허리가 늘고 긴 체형 어...,NaN,"가슴, 개, 덜하다, 듬직하다, 숨기다, 어깨, 여전하다, 치마, 태...",-,"글로리, 말로, 말, 모, 글로리, 핏, 이쁜, 거, 두말하면, 입, 아파요, 허리..."
1,예약배송이라 쇼룸에서 사이즈 입어보고 샀는데 좀.. 다른 거 같아요 배송된게 좀 더...,예약배송이라 쇼룸에서 사이즈 입어보고 샀는데 좀 다른 거 같아요 배송된 게 좀 더 ...,NaN,"대신, 민소매, 배송, 부담스럽다, 불편하다, 사이즈, 소재, 쇼룸, ...",배송,"예, 약, 배송, 이라, 쇼룸, 에서, 사이즈, 입어보다, 사다, 좀, 다른, 거,..."
2,화사해보이고 예쁘지만 기장이 조금만 더 길었느면 좋겠음\n많이 짧아서 좀 부담스러울수도,화사해 보이고 예쁘지만 기장이 조금만 더 길었으면 좋겠음 많이 짧아서 좀 부담스러울 수도,"기장, 짧다, 화사하다","기장, 길다, 보이, 부담스럽다, 수도, 짧다, 화사하다",-,"화사해, 보이고, 예쁘지만, 기장이, 조금만, 더, 길었으면, 좋겠음, 많이, 짧아..."
3,두겹이라 비치지도 않고 \n완전 탄탄합니다\n올해 원피스중 제일 이쁜듯요\n너무 맘...,두 겹이라 비치지도 않고 정말 탄탄합니다 올해 원피스 중 제일 이쁜 듯요 너무 맘에...,NaN,"겹, 들어서다, 비치다, 올해, 원피스, 칼라, 탄탄하다",-,"두, 겹이라, 비치지도, 않고, 정말, 탄탄합니다, 올해, 원피스, 중, 제일, 이..."
4,재질이 탄탄해서 흰색임에도 비침이 거의 없어서 좋았어요. 노란기나 푸른기 없이 깨끗...,재질이 탄탄해서 흰색임에도 비침이 거의 없어서 좋았어요 노란 기나 푸른 기 없이 깨...,NaN,"깨끗하다, 노랗다, 사이즈, 색깔, 신경, 재질, 정석, 탄탄하다, 흰색",-,"재질이, 탄탄해서, 흰색임에도, 비침이, 거의, 없어서, 좋았어요, 노란, 기나, ..."
...,...,...,...,...,...,...
88,작은 제 키에는 살짝 긴 감이 있지만 무척 예쁘고 편합니다.,작은 제 키에는 살짝 긴 감이 있지만 무척 예쁘고 편합니다,NaN,"무척, 에는, 작다, 키, 편하다",-,"작은, 제, 키에는, 살짝, 긴, 감이, 있지만, 무척, 예쁘고, 편합니다"
89,딱 붙는 핏 찾았었는데 생각한대로에요 여기저기 입기 좋아요,딱 붙는 핏 찾았었는데 생각한 대로에요 여기저기 입기 좋아요,대로,"대로, 붙다, 여기저기, 찾다",-,"딱, 붙는, 핏, 찾았었는데, 생각한, 대로에요, 여기저기, 입기, 좋아요"
90,믿고 사는 글로니 핏 너무 예뻐요 완전 깔끔해요 다른색도 살거에요,믿고 사는 글로리 핏 너무 예뻐요 정말 깔끔해요 다른 색도 살 거예요,살다,"글로리, 깔끔하다, 믿다, 살다, 색도",-,"믿고, 사는, 글로리, 핏, 너무, 예뻐요, 정말, 깔끔해요, 다른, 색도, 살, 거예요"
91,블랙도 사고 싶을 정도류 예쁜데 너무짧음 소재는 너무 얇지 않아 좋아요,블랙도 사고 싶을 정도류 예쁜데 너무 짧음 소재는 너무 얇지 않아 좋아요,"짧다, 소재","류, 블랙, 사고, 소재, 얇다, 짧다",-,"블랙도, 사고, 싶을, 정도류, 예쁜데, 너무, 짧음, 소재는, 너무, 얇지, 않아..."


In [572]:
# 'delivery' 열의 값이 '배송'인 행만 필터링하여 'Processed_Review', 'Processed_Review_Okt'를 확인
delivery_filtered = df[df['delivery'] == '배송'][['Processed_Review', 'common_keywords_3_plus', 'delivery', 'Processed_Review_Okt']]
delivery_filtered

,Processed_Review,common_keywords_3_plus,delivery,Processed_Review_Okt
1,예약배송이라 쇼룸에서 사이즈 입어보고 샀는데 좀 다른 거 같아요 배송된 게 좀 더 ...,"대신, 민소매, 배송, 부담스럽다, 불편하다, 사이즈, 소재, 쇼룸, ...",배송,"예, 약, 배송, 이라, 쇼룸, 에서, 사이즈, 입어보다, 사다, 좀, 다른, 거,..."


In [573]:
# 빠르다/느리다 관련 키워드 리스트
fast_keywords = ["빨", "로켓", "반나절", "일찍"]  # 예외는 밑에서 따로 처리
slow_keywords = ["느리다", "예약배송", "예약", "걸리다", "출고", "늦어지다", "아쉽다", "늦다", "감점", "넘다", "늦", "지연"]

# 원본 DataFrame을 수정하거나 슬라이스된 데이터프레임을 복사하여 작업
delivery_related_rows = delivery_filtered.copy()

# 'speed' 열 추가하여 빠르다/느리다 분류
def classify_speed(x):
    if not isinstance(x, str):
        return '만족하다'
    
    # '빠르다' 분류 조건
    if (
        any(keyword in x for keyword in fast_keywords) or
        ('빠르다' in x and ('오다' in x or '배송' in x)) or
        ('오다' in x and ('하루' in x or '빨리' in x)) or
        ('하루' in x and ('온' in x or '만에' in x)) or
        ('깔끔하다' in x and '오다' in x) or
        ('칼' in x and ('오다' in x or '배송' in x))
    ):
        return '빠르다'
    
    # '느리다' 분류 조건
    if (
        any(keyword in x for keyword in slow_keywords) or
        ('이틀' in x and ('온' in x or '만에' in x or '오다' in x)) or
        ('일주일' in x and ('온' in x or '만에' in x or '오다' in x))
    ):
        return '느리다'
    
    return '만족하다'

delivery_related_rows['speed'] = delivery_related_rows['Processed_Review_Okt'].apply(classify_speed)
delivery_related_rows

,Processed_Review,common_keywords_3_plus,delivery,Processed_Review_Okt,speed
1,예약배송이라 쇼룸에서 사이즈 입어보고 샀는데 좀 다른 거 같아요 배송된 게 좀 더 ...,"대신, 민소매, 배송, 부담스럽다, 불편하다, 사이즈, 소재, 쇼룸, ...",배송,"예, 약, 배송, 이라, 쇼룸, 에서, 사이즈, 입어보다, 사다, 좀, 다른, 거,...",만족하다


In [574]:
# 행 번호가 n인 ~~열의 값 출력
# print(delivery_filtered.loc[186, 'Processed_Review'])
# print(delivery_filtered.loc[470, 'Processed_Review_Okt'])   # Processed_Review_Okt
# print(delivery_filtered.loc[921, 'common_keywords_3_plus'])

In [575]:
# 빠르다/느리다의 개수와 비율 계산
speed_nums = delivery_related_rows['speed'].value_counts()
total_nums = len(delivery_filtered)

# '느리다'가 없으면 '만족하다'로 대체
if '느리다' not in speed_nums.index:
    speed_nums['만족하다'] = total_nums - speed_nums.get('빠르다', 0)

# 비율 계산
speed_ratios = (speed_nums / total_nums) * 100
speed_ratios_format = speed_ratios.apply(lambda x: f"{x:.2f}%")

# '느리다'가 있는지 확인하여 출력할 결과 설정
if '느리다' in speed_nums.index:
    # '빠르다'와 '느리다'의 비율만 필터링
    filter_speed_ratio = speed_ratios_format[speed_ratios_format.index.isin(['빠르다', '느리다'])]
else:
    # '빠르다'와 '만족하다'의 비율만 필터링
    filter_speed_ratio = speed_ratios_format[speed_ratios_format.index.isin(['빠르다', '만족하다'])]

# 결과 출력
print(f"배송 비율 (배송 관련 행 수 {total_nums}개 대비):")
print(filter_speed_ratio)

배송 비율 (배송 관련 행 수 1개 대비):
speed
만족하다    100.00%
Name: count, dtype: object


In [11]:
# 위에 비율: 빠르다,느리다 행 수 / 배송 관련 행 수
# 밑에 비율: 빠르다,느리다 행 수 / csv 파일 전체 행 수

# CSV 파일 전체 행 수 계산
total_row_nums = len(df)

# '빠르다'와 '느리다'의 개수 계산
speed_nums = delivery_related_rows['speed'].value_counts()

# 빠르다, 느리다 비율을 CSV 전체 행 수로 계산
all_speed_ratios = (speed_nums / total_row_nums) * 100

# 결과를 소수점 둘째 자리까지 포맷팅하고 % 추가
all_speed_ratios_format = all_speed_ratios.apply(lambda x: f"{x:.2f}%")

# '빠르다'와 '느리다'의 비율만 필터링 ("만족해요"는 제외)
filter_all_speed_ratio = all_speed_ratios_format[all_speed_ratios_format.index.isin(['빠르다', '느리다'])]

# 결과 출력
print(f"배송 비율 (전체 행 수 {total_row_nums}개 대비):")
print(filter_all_speed_ratio)

배송 비율 (전체 행 수 879개 대비):
speed
빠르다    11.04%
느리다     0.68%
Name: count, dtype: object


## 색감 - Okt만 적용 (기본형 변환 x)

In [576]:
file_path = './해시태그병합/Deep One Tuck Sweat Shorts [Grey]_keywords.csv'

# 파일 이름 추출
file_name = file_path.split('/')[-1].split('.')[0]   # 마지막 슬래시(/)와 .csv를 기준으로 파일 이름 추출

# 파일 읽기
df = pd.read_csv(file_path)
df

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus
0,만족해요\n크게 입는거 추천\n농구선수가 입는 바지 느낌 나용,만족해요 크게 입는 거 추천 농구선수가 입는 바지 느낌 나용,"추천, 바지, 크게","나용, 농구, 만족하다, 바지, 선수, 추천, 크게"
1,와이드핏으로 크게 입을수있어요 흐흐 시웡해요 흐흐,와이드 핏으로 크게 입을 수 있어요 흐흐 시웡해요 흐흐,크게,"시웡해, 와이드, 크게, 핏, 흐흐"
2,사이즈가 클 줄 알았는데 생각보다 안 크고 허리가 짱짱해서 좋습니댜 근데 걷다보면 ...,사이즈가 클 줄 알았는데 생각보다 안 크고 허리가 짱짱해서 좋습니다 근데 걷다 보면...,허리,"내려가다, 사이즈, 알다, 접다, 짱짱하다, 크다, 허리"
3,몸무게가 많이 나가 작진 않을까 했는데 너무 잘 맞내요,몸무게가 많이 나가 작진 않을까 했는데 너무 잘 맞네요,NaN,"맞다, 몸무게, 작다"
4,첨엔 생각보다 안이뻐보여서 실망했는데 입다보니 푹 빠져버렸어요 너무좋아요,첨엔 생각보다 안 이뻐 보여서 실망했는데 입다 보니 푹 빠져버렸어요 너무 좋아요,NaN,"버리다, 빠지다, 실망하다, 여서, 첨"
...,...,...,...,...
700,카키색 사고 마음에 들어서 회색도 추가로 구매했어요😀 일단 편하구 메쉬소재라 한여름...,카키색 사고 마음에 들어서 회색도 추가로 구매했어요 일단 편하고 메시 소재라 한여름...,NaN,"가볍다, 넉넉하다, 버뮤다, 분들, 사이즈, 여성, 편하다"
701,운동할때 입으려고 샀어요! 인기 있는건 이유가 있네요\n싸게 잘 사서 입고있습니다 ㅎㅎ,운동할 때 입으려고 샀어요 인기 있는 건 이유가 있네요 싸게 잘 사서 입고 있습니다,NaN,"싸다, 운동, 이유, 인기"
702,요즘 유행이라서 사봤는데 진짜진짜 넘 예쁘고 넘 편해여 오버핏 상의랑 코디하면 예쁘...,요즘 유행이라서 사봤는데 진짜 진짜 너무 예쁘고 너무 편해요 오버핏 상의랑 코디하면...,NaN,"사보다, 상의, 오버핏, 유행, 코디, 편하다"
703,너무 이뻐요. 딱 제 스타일이에요. 잘 산것 같아요. 강추,너무 이뻐요 딱 제 스타일이에요 잘 산 것 같아요 강추,NaN,"강추, 스타일"


In [577]:
# common_keywords_3_plus 키워드 버핏 -> 오버핏으로
df['common_keywords_3_plus'] = df['common_keywords_3_plus'].apply(lambda x: ', '.join(['오버핏' if keyword.strip() == '버핏' else keyword for keyword in str(x).split(',')]) if pd.notnull(x) else x)
df

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus
0,만족해요\n크게 입는거 추천\n농구선수가 입는 바지 느낌 나용,만족해요 크게 입는 거 추천 농구선수가 입는 바지 느낌 나용,"추천, 바지, 크게","나용, 농구, 만족하다, 바지, 선수, 추천, 크게"
1,와이드핏으로 크게 입을수있어요 흐흐 시웡해요 흐흐,와이드 핏으로 크게 입을 수 있어요 흐흐 시웡해요 흐흐,크게,"시웡해, 와이드, 크게, 핏, 흐흐"
2,사이즈가 클 줄 알았는데 생각보다 안 크고 허리가 짱짱해서 좋습니댜 근데 걷다보면 ...,사이즈가 클 줄 알았는데 생각보다 안 크고 허리가 짱짱해서 좋습니다 근데 걷다 보면...,허리,"내려가다, 사이즈, 알다, 접다, 짱짱하다, 크다, 허리"
3,몸무게가 많이 나가 작진 않을까 했는데 너무 잘 맞내요,몸무게가 많이 나가 작진 않을까 했는데 너무 잘 맞네요,NaN,"맞다, 몸무게, 작다"
4,첨엔 생각보다 안이뻐보여서 실망했는데 입다보니 푹 빠져버렸어요 너무좋아요,첨엔 생각보다 안 이뻐 보여서 실망했는데 입다 보니 푹 빠져버렸어요 너무 좋아요,NaN,"버리다, 빠지다, 실망하다, 여서, 첨"
...,...,...,...,...
700,카키색 사고 마음에 들어서 회색도 추가로 구매했어요😀 일단 편하구 메쉬소재라 한여름...,카키색 사고 마음에 들어서 회색도 추가로 구매했어요 일단 편하고 메시 소재라 한여름...,NaN,"가볍다, 넉넉하다, 버뮤다, 분들, 사이즈, 여성, 편하다"
701,운동할때 입으려고 샀어요! 인기 있는건 이유가 있네요\n싸게 잘 사서 입고있습니다 ㅎㅎ,운동할 때 입으려고 샀어요 인기 있는 건 이유가 있네요 싸게 잘 사서 입고 있습니다,NaN,"싸다, 운동, 이유, 인기"
702,요즘 유행이라서 사봤는데 진짜진짜 넘 예쁘고 넘 편해여 오버핏 상의랑 코디하면 예쁘...,요즘 유행이라서 사봤는데 진짜 진짜 너무 예쁘고 너무 편해요 오버핏 상의랑 코디하면...,NaN,"사보다, 상의, 오버핏, 유행, 코디, 편하다"
703,너무 이뻐요. 딱 제 스타일이에요. 잘 산것 같아요. 강추,너무 이뻐요 딱 제 스타일이에요 잘 산 것 같아요 강추,NaN,"강추, 스타일"


In [578]:
# 색감 관련 단어 리스트
color_keywords = ["색감", "색상", "색", "밝다", "어둡다", "밝은", "어두운", "무채색", "크림색", "흰색", "하얀색", "화이트", "빨간색", "회색", "검은색", 
                  "보랏빛", "네이비", "브라운", "베이지", "보라색", "검정", "블랙", "그레이", "블루", "초록색", "그린", "진하다", "오묘하다"]

# 새로운 열 '색감'을 추가하여 단어가 포함된 경우 '색감'으로 분류 (정확한 단어 매칭)
df['color'] = df['common_keywords_3_plus'].apply(
    lambda x: '색감' if isinstance(x, str) and any(keyword.strip() in color_keywords for keyword in x.split(',')) else '-'
)

df

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus,color
0,만족해요\n크게 입는거 추천\n농구선수가 입는 바지 느낌 나용,만족해요 크게 입는 거 추천 농구선수가 입는 바지 느낌 나용,"추천, 바지, 크게","나용, 농구, 만족하다, 바지, 선수, 추천, 크게",-
1,와이드핏으로 크게 입을수있어요 흐흐 시웡해요 흐흐,와이드 핏으로 크게 입을 수 있어요 흐흐 시웡해요 흐흐,크게,"시웡해, 와이드, 크게, 핏, 흐흐",-
2,사이즈가 클 줄 알았는데 생각보다 안 크고 허리가 짱짱해서 좋습니댜 근데 걷다보면 ...,사이즈가 클 줄 알았는데 생각보다 안 크고 허리가 짱짱해서 좋습니다 근데 걷다 보면...,허리,"내려가다, 사이즈, 알다, 접다, 짱짱하다, 크다, 허리",-
3,몸무게가 많이 나가 작진 않을까 했는데 너무 잘 맞내요,몸무게가 많이 나가 작진 않을까 했는데 너무 잘 맞네요,NaN,"맞다, 몸무게, 작다",-
4,첨엔 생각보다 안이뻐보여서 실망했는데 입다보니 푹 빠져버렸어요 너무좋아요,첨엔 생각보다 안 이뻐 보여서 실망했는데 입다 보니 푹 빠져버렸어요 너무 좋아요,NaN,"버리다, 빠지다, 실망하다, 여서, 첨",-
...,...,...,...,...,...
700,카키색 사고 마음에 들어서 회색도 추가로 구매했어요😀 일단 편하구 메쉬소재라 한여름...,카키색 사고 마음에 들어서 회색도 추가로 구매했어요 일단 편하고 메시 소재라 한여름...,NaN,"가볍다, 넉넉하다, 버뮤다, 분들, 사이즈, 여성, 편하다",-
701,운동할때 입으려고 샀어요! 인기 있는건 이유가 있네요\n싸게 잘 사서 입고있습니다 ㅎㅎ,운동할 때 입으려고 샀어요 인기 있는 건 이유가 있네요 싸게 잘 사서 입고 있습니다,NaN,"싸다, 운동, 이유, 인기",-
702,요즘 유행이라서 사봤는데 진짜진짜 넘 예쁘고 넘 편해여 오버핏 상의랑 코디하면 예쁘...,요즘 유행이라서 사봤는데 진짜 진짜 너무 예쁘고 너무 편해요 오버핏 상의랑 코디하면...,NaN,"사보다, 상의, 오버핏, 유행, 코디, 편하다",-
703,너무 이뻐요. 딱 제 스타일이에요. 잘 산것 같아요. 강추,너무 이뻐요 딱 제 스타일이에요 잘 산 것 같아요 강추,NaN,"강추, 스타일",-


In [579]:
# 'color'열에 '색감'인 행 있으면, 'Processed_Review'열에서 okt 적용 (기본형 변환 x)
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

# 'Processed_Review_Okt' 열을 생성하고 기본값으로 'Processed_Review' 열 복사
df['Processed_Review_Okt'] = df['Processed_Review']

# 'color'가 '색감'인 행에 대한 필터 생성
color_filter = df['color'] == '색감'

# '색감'인 행에 대해 Okt 형태소 분석 적용 후, 결과를 문자열로 변환 (기본형 변환 x)
df.loc[color_filter, 'Processed_Review_Okt'] = df.loc[color_filter, 'Processed_Review'].apply(lambda x: ' '.join(okt.morphs(x, stem=False)))

# 'Processed_Review_Okt' 열에서 값을 쉼표로 연결
df['Processed_Review_Okt'] = df['Processed_Review_Okt'].apply(
    lambda x: ', '.join(x.split()) if isinstance(x, str) else x
)

df

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus,color,Processed_Review_Okt
0,만족해요\n크게 입는거 추천\n농구선수가 입는 바지 느낌 나용,만족해요 크게 입는 거 추천 농구선수가 입는 바지 느낌 나용,"추천, 바지, 크게","나용, 농구, 만족하다, 바지, 선수, 추천, 크게",-,"만족해요, 크게, 입는, 거, 추천, 농구선수가, 입는, 바지, 느낌, 나용"
1,와이드핏으로 크게 입을수있어요 흐흐 시웡해요 흐흐,와이드 핏으로 크게 입을 수 있어요 흐흐 시웡해요 흐흐,크게,"시웡해, 와이드, 크게, 핏, 흐흐",-,"와이드, 핏으로, 크게, 입을, 수, 있어요, 흐흐, 시웡해요, 흐흐"
2,사이즈가 클 줄 알았는데 생각보다 안 크고 허리가 짱짱해서 좋습니댜 근데 걷다보면 ...,사이즈가 클 줄 알았는데 생각보다 안 크고 허리가 짱짱해서 좋습니다 근데 걷다 보면...,허리,"내려가다, 사이즈, 알다, 접다, 짱짱하다, 크다, 허리",-,"사이즈가, 클, 줄, 알았는데, 생각보다, 안, 크고, 허리가, 짱짱해서, 좋습니다..."
3,몸무게가 많이 나가 작진 않을까 했는데 너무 잘 맞내요,몸무게가 많이 나가 작진 않을까 했는데 너무 잘 맞네요,NaN,"맞다, 몸무게, 작다",-,"몸무게가, 많이, 나가, 작진, 않을까, 했는데, 너무, 잘, 맞네요"
4,첨엔 생각보다 안이뻐보여서 실망했는데 입다보니 푹 빠져버렸어요 너무좋아요,첨엔 생각보다 안 이뻐 보여서 실망했는데 입다 보니 푹 빠져버렸어요 너무 좋아요,NaN,"버리다, 빠지다, 실망하다, 여서, 첨",-,"첨엔, 생각보다, 안, 이뻐, 보여서, 실망했는데, 입다, 보니, 푹, 빠져버렸어요..."
...,...,...,...,...,...,...
700,카키색 사고 마음에 들어서 회색도 추가로 구매했어요😀 일단 편하구 메쉬소재라 한여름...,카키색 사고 마음에 들어서 회색도 추가로 구매했어요 일단 편하고 메시 소재라 한여름...,NaN,"가볍다, 넉넉하다, 버뮤다, 분들, 사이즈, 여성, 편하다",-,"카키색, 사고, 마음에, 들어서, 회색도, 추가로, 구매했어요, 일단, 편하고, 메..."
701,운동할때 입으려고 샀어요! 인기 있는건 이유가 있네요\n싸게 잘 사서 입고있습니다 ㅎㅎ,운동할 때 입으려고 샀어요 인기 있는 건 이유가 있네요 싸게 잘 사서 입고 있습니다,NaN,"싸다, 운동, 이유, 인기",-,"운동할, 때, 입으려고, 샀어요, 인기, 있는, 건, 이유가, 있네요, 싸게, 잘,..."
702,요즘 유행이라서 사봤는데 진짜진짜 넘 예쁘고 넘 편해여 오버핏 상의랑 코디하면 예쁘...,요즘 유행이라서 사봤는데 진짜 진짜 너무 예쁘고 너무 편해요 오버핏 상의랑 코디하면...,NaN,"사보다, 상의, 오버핏, 유행, 코디, 편하다",-,"요즘, 유행이라서, 사봤는데, 진짜, 진짜, 너무, 예쁘고, 너무, 편해요, 오버핏..."
703,너무 이뻐요. 딱 제 스타일이에요. 잘 산것 같아요. 강추,너무 이뻐요 딱 제 스타일이에요 잘 산 것 같아요 강추,NaN,"강추, 스타일",-,"너무, 이뻐요, 딱, 제, 스타일이에요, 잘, 산, 것, 같아요, 강추"


In [580]:
# 'color'에서 '색감'인 행만 필터링하여 'Processed_Review', 'Processed_Review_Okt'를 확인
color_filtered = df[df['color'] == '색감'][['Processed_Review', 'common_keywords_3_plus', 'color', 'Processed_Review_Okt']]
color_filtered

,Processed_Review,common_keywords_3_plus,color,Processed_Review_Okt
52,검은색도 하나 사려고요 만족합니다 길이 딱 좋아요,"검은색, 길이, 만족하다, 하나",색감,"검은색, 도, 하나, 사려고요, 만족합니다, 길이, 딱, 좋아요"
64,저는 상체에 비해 하체가 좀 있는 편이라 반바지를 잘 안 입는데 이건 후기가 괜찮아...,"만족하다, 반바지, 블랙, 상체, 신체, 이건, 추천",색감,"저, 는, 상체, 에, 비해, 하체, 가, 좀, 있는, 편이, 라, 반바지, 를, ..."
68,색이랑 두께감 다 좋아요 굿굿 엄청 더울 때 빼고 휘뚜루마뚜루 잘 입고 다닐 거 같아요,"굿굿, 덥다, 두께감, 빼다, 색, 휘뚜루마뚜루",색감,"색, 이랑, 두께감, 다, 좋아요, 굿굿, 엄청, 더울, 때, 빼고, 휘뚜루마뚜루,..."
87,s랑 m이랑 되게 고민 많이 했는데 m은 너무 클 것 같아서 s로 샀더니 딱 괜찮은...,"검정, 고민, 되게, 들어서다",색감,"s, 랑, m, 이랑, 되게, 고민, 많이, 했는데, m, 은, 너무, 클, 것, ..."
99,길이 좋아요 핏도 예뻐요 다른 색으로 또 사고 싶어요 여름 동안 이것만 입을 것 같아요,"길이, 색, 여름",색감,"길이, 좋아요, 핏, 도, 예뻐요, 다른, 색, 으로, 또, 사고, 싶어요, 여름,..."
106,디자인 무난한 디자인에 모두 다 아는 회색 트레이닝 바지입니다 그러다 보니 활용도 ...,"디자인, 라고, 바지, 사이즈, 색상",색감,"디자인, 무난, 한, 디자인, 에, 모두, 다, 아는, 회색, 트레이닝, 바지, 입..."
109,기장감도 딱 예쁘고 핏이나 색상도 맘에 들어요 역시 인기 많은 이유가 있네요,"기장감, 많다, 색상, 이유, 인기",색감,"기장감, 도, 딱, 예쁘고, 핏, 이나, 색상, 도, 맘, 에, 들어요, 역시, 인..."
110,S 시켜서 짧을까 걱정했는데 무릎 덮는 기장에 색이랑 통 다 마음에 들어요,"걱정, 기장, 덮다, 무릎, 색, 짧다",색감,"S, 시켜서, 짧을까, 걱정, 했는데, 무릎, 덮는, 기장, 에, 색, 이랑, 통,..."
126,회색이고 무난하니 좋아요 어떤 색을 매치하던 예뻐요,"매치, 무난, 색, 어떻다, 이고, 회색",색감,"회색, 이고, 무난, 하니, 좋아요, 어떤, 색, 을, 매치, 하던, 예뻐요"
157,핏도 괜찮고 색감도 좋아서 마음에 드는 상품입니다,"상품, 색감, 핏",색감,"핏, 도, 괜찮고, 색감, 도, 좋아서, 마음, 에, 드는, 상품, 입니다"


In [581]:
# 생각보다 밝아요, 생각보다 어두워요 관련 단어 리스트
light_keywords = ["밝다", "밝은", "크림색", "흰색", "하얀색", "화이트", "빨간색", "베이지"]
dark_keywords = ["어둡다", "어두운", "회색", "검은색", "보랏빛", "네이비", "브라운", "보라색", "검정", "블랙", "그레이", "블루", "초록색", "그린"]

# 원본 DataFrame을 수정하거나 슬라이스된 데이터프레임을 복사하여 작업
color_related_rows = color_filtered.copy()

# brightness 열 추가
color_related_rows.loc[:, 'brightness'] = color_related_rows['Processed_Review_Okt'].apply(
    lambda x: '생각보다 밝아요' if isinstance(x, str) and any(keyword in x for keyword in light_keywords)
              else ('생각보다 어두워요' if isinstance(x, str) and any(keyword in x for keyword in dark_keywords) else '만족해요')
)

color_related_rows

,Processed_Review,common_keywords_3_plus,color,Processed_Review_Okt,brightness
52,검은색도 하나 사려고요 만족합니다 길이 딱 좋아요,"검은색, 길이, 만족하다, 하나",색감,"검은색, 도, 하나, 사려고요, 만족합니다, 길이, 딱, 좋아요",생각보다 어두워요
64,저는 상체에 비해 하체가 좀 있는 편이라 반바지를 잘 안 입는데 이건 후기가 괜찮아...,"만족하다, 반바지, 블랙, 상체, 신체, 이건, 추천",색감,"저, 는, 상체, 에, 비해, 하체, 가, 좀, 있는, 편이, 라, 반바지, 를, ...",생각보다 어두워요
68,색이랑 두께감 다 좋아요 굿굿 엄청 더울 때 빼고 휘뚜루마뚜루 잘 입고 다닐 거 같아요,"굿굿, 덥다, 두께감, 빼다, 색, 휘뚜루마뚜루",색감,"색, 이랑, 두께감, 다, 좋아요, 굿굿, 엄청, 더울, 때, 빼고, 휘뚜루마뚜루,...",만족해요
87,s랑 m이랑 되게 고민 많이 했는데 m은 너무 클 것 같아서 s로 샀더니 딱 괜찮은...,"검정, 고민, 되게, 들어서다",색감,"s, 랑, m, 이랑, 되게, 고민, 많이, 했는데, m, 은, 너무, 클, 것, ...",생각보다 어두워요
99,길이 좋아요 핏도 예뻐요 다른 색으로 또 사고 싶어요 여름 동안 이것만 입을 것 같아요,"길이, 색, 여름",색감,"길이, 좋아요, 핏, 도, 예뻐요, 다른, 색, 으로, 또, 사고, 싶어요, 여름,...",만족해요
106,디자인 무난한 디자인에 모두 다 아는 회색 트레이닝 바지입니다 그러다 보니 활용도 ...,"디자인, 라고, 바지, 사이즈, 색상",색감,"디자인, 무난, 한, 디자인, 에, 모두, 다, 아는, 회색, 트레이닝, 바지, 입...",생각보다 어두워요
109,기장감도 딱 예쁘고 핏이나 색상도 맘에 들어요 역시 인기 많은 이유가 있네요,"기장감, 많다, 색상, 이유, 인기",색감,"기장감, 도, 딱, 예쁘고, 핏, 이나, 색상, 도, 맘, 에, 들어요, 역시, 인...",만족해요
110,S 시켜서 짧을까 걱정했는데 무릎 덮는 기장에 색이랑 통 다 마음에 들어요,"걱정, 기장, 덮다, 무릎, 색, 짧다",색감,"S, 시켜서, 짧을까, 걱정, 했는데, 무릎, 덮는, 기장, 에, 색, 이랑, 통,...",만족해요
126,회색이고 무난하니 좋아요 어떤 색을 매치하던 예뻐요,"매치, 무난, 색, 어떻다, 이고, 회색",색감,"회색, 이고, 무난, 하니, 좋아요, 어떤, 색, 을, 매치, 하던, 예뻐요",생각보다 어두워요
157,핏도 괜찮고 색감도 좋아서 마음에 드는 상품입니다,"상품, 색감, 핏",색감,"핏, 도, 괜찮고, 색감, 도, 좋아서, 마음, 에, 드는, 상품, 입니다",만족해요


In [582]:
# 행 번호가 n인 Processed_Review_Okt 열의 값 출력
# print(color_filtered.loc[12, 'Processed_Review_Okt'])

In [583]:
# '생각보다 밝아요'와 '생각보다 어두워요'의 개수 계산
color_counts = color_related_rows['brightness'].value_counts()

# 전체 행 수
total_counts = len(color_related_rows)

# '생각보다 밝아요'와 '생각보다 어두워요'의 비율 계산
color_ratios = (color_counts / total_counts) * 100

# 결과를 소수점 둘째 자리까지 포맷팅하고 % 추가
color_ratios_formatted = color_ratios.apply(lambda x: f"{x:.2f}%")

# '생각보다 밝아요', '만족해요', '생각보다 어두워요' 순으로 인덱스를 정렬
sorted_order = ['생각보다 밝아요', '만족해요', '생각보다 어두워요']

# 인덱스 순서를 지정한 대로 맞춤
color_ratios_sorted = color_ratios_formatted.reindex(sorted_order)

# 결과 출력
print(f"색감 비율 (색감 관련 행 수 {total_counts}개 대비):")
print(color_ratios_sorted)

색감 비율 (색감 관련 행 수 32개 대비):
brightness
생각보다 밝아요        NaN
만족해요         40.62%
생각보다 어두워요    59.38%
Name: count, dtype: object


In [19]:
# '~~' 단어가 포함된 행 찾기
contain_word = color_related_rows[color_related_rows['Processed_Review_Okt'].str.contains('밝은')]
contain_word

,Processed_Review,common_keywords_3_plus,color,Processed_Review_Okt,brightness
512,아디다스 트레이닝복이랑 깔 맞춤으로 딱 좋네요 밝은 프린팅으로 여름에 시원해 보이고...,"맞춤, 밝다, 보이, 시원하다, 아디다스, 여름, 트레이닝복, 프린팅",색감,"아디다스, 트레이닝복, 이랑, 깔, 맞춤, 으로, 딱, 좋네요, 밝은, 프린팅, 으...",생각보다 밝아요


## 한 파일씩 적용할 전체 함수

In [567]:
# 배송 전체 함수
from konlpy.tag import Okt
import re

def process_delivery_data(file_path):
    # 파일 이름 추출
    file_name = file_path.split('/')[-1].split('.')[0]
    
    # 파일 읽기
    df = pd.read_csv(file_path)
    
    # 후처리: '정 사이즈' -> '정사이즈', '온 버핏' -> '오버핏'
    df['Processed_Review'] = df['Processed_Review'].apply(
        lambda x: x.replace('정 사이즈', '정사이즈').replace('온 버핏', '오버핏') if pd.notnull(x) else x
    )
    
    # '버핏' -> '오버핏' 변경 in common_keywords_3_plus
    df['common_keywords_3_plus'] = df['common_keywords_3_plus'].apply(
        lambda x: ', '.join(['오버핏' if keyword.strip() == '버핏' else keyword for keyword in str(x).split(',')]) if pd.notnull(x) else x
    )
    
    # 배송 관련 단어 리스트 (이 외에는 예외 처리)
    shipping_keywords = ["배송", "느리다", "예약배송", "예약", "출고"]
    
    # 'delivery' 여부 분류를 위한 정확한 키워드 매칭 함수
    def is_delivery(keywords):
        keywords = [k.strip() for k in keywords.split(',')]
        
        # 조건 1: shipping_keywords에 정확히 일치하는 키워드가 있는지
        if any(k in shipping_keywords for k in keywords):
            return '배송'
        
        # 조건 2: '깔끔하다'와 정확히 '배송' 또는 '오다'가 함께 있는지
        if '깔끔하다' in keywords and ('배송' in keywords or '오다' in keywords):
            return '배송'
        
        # 조건 3: '빨리'와 '오다'가 함께 있는지
        if '빨리' in keywords and '오다' in keywords:
            return '배송'
        
        # 조건 4: '하루', '이틀', '일주일' 중 하나와 '오다', '온', '만에' 중 하나가 함께 있는지
        if any(k in ['하루', '이틀', '일주일'] for k in keywords) and any(k in ['오다', '온', '만에'] for k in keywords):
            return '배송'
        
        # 조건 5: '빠르다'와 '오다' 또는 '배송'이 함께 있는지
        if '빠르다' in keywords and ('오다' in keywords or '배송' in keywords):
            return '배송'
        
        # 조건 6: '기다리다'와 '오다' 또는 '배송'이 함께 있는지
        if '기다리다' in keywords and ('오다' in keywords or '배송' in keywords):
            return '배송'
        
        return '-'
    
    # '배송' 여부 분류
    df['delivery'] = df['common_keywords_3_plus'].apply(
        lambda x: is_delivery(x) if isinstance(x, str) else '-'
    )
    
    # Okt 형태소 분석기 초기화
    okt = Okt()
    
    # 'Processed_Review_Okt' 열 생성 및 배송 관련 행에 대해 기본형 변환
    df['Processed_Review_Okt'] = df['Processed_Review']
    df.loc[df['delivery'] == '배송', 'Processed_Review_Okt'] = df.loc[df['delivery'] == '배송', 'Processed_Review'].apply(
        lambda x: ' '.join([word[0] for word in okt.pos(x, stem=True)]) if isinstance(x, str) else x
    )
    
    # 쉼표로 연결된 'Processed_Review_Okt'
    df['Processed_Review_Okt'] = df['Processed_Review_Okt'].apply(
        lambda x: ', '.join(x.split()) if isinstance(x, str) else x
    )
    
    # '배송' 관련 행 필터링
    delivery_filtered = df[df['delivery'] == '배송'][['Processed_Review', 'common_keywords_3_plus', 'delivery', 'Processed_Review_Okt']]
    
    # 빠르다/느리다 관련 키워드 리스트
    fast_keywords = ["빨", "로켓", "반나절", "일찍"]
    slow_keywords = ["느리다", "예약배송", "예약", "걸리다", "출고", "늦어지다", "아쉽다", "늦다", "감점", "넘다", "늦", "지연"]
    
    # 'speed' 열 추가하여 빠르다/느리다 분류
    def classify_speed(x):
        if not isinstance(x, str):
            return '만족하다'
        
        # '빠르다' 분류 조건
        if (
            any(keyword in x for keyword in fast_keywords) or
            ('빠르다' in x and ('오다' in x or '배송' in x)) or
            ('오다' in x and ('하루' in x or '빨리' in x)) or
            ('하루' in x and ('온' in x or '만에' in x)) or
            ('깔끔하다' in x and '오다' in x) or
            ('칼' in x and ('오다' in x or '배송' in x))
        ):
            return '빠르다'
        
        # '느리다' 분류 조건
        if (
            any(keyword in x for keyword in slow_keywords) or
            ('이틀' in x and ('온' in x or '만에' in x or '오다' in x)) or
            ('일주일' in x and ('온' in x or '만에' in x or '오다' in x))
        ):
            return '느리다'
        
        return '만족하다'
    
    delivery_filtered['speed'] = delivery_filtered['Processed_Review_Okt'].apply(classify_speed)
    
    # 빠르다/느리다의 개수와 비율 계산
    speed_nums = delivery_filtered['speed'].value_counts()
    total_nums = len(delivery_filtered)
    
    # '느리다'가 없으면 '만족하다'로 대체
    if '느리다' not in speed_nums.index:
        speed_nums['만족하다'] = total_nums - speed_nums.get('빠르다', 0)
    
    # 비율 계산
    speed_ratios = (speed_nums / total_nums) * 100
    speed_ratios_format = speed_ratios.apply(lambda x: f"{x:.2f}%")
    
    # '느리다'가 있는지 확인하여 출력할 결과 설정
    if '느리다' in speed_nums.index:
        # '빠르다'와 '느리다'의 비율만 필터링
        filter_speed_ratio = speed_ratios_format[speed_ratios_format.index.isin(['빠르다', '느리다'])]
    else:
        # '빠르다'와 '만족하다'의 비율만 필터링
        filter_speed_ratio = speed_ratios_format[speed_ratios_format.index.isin(['빠르다', '만족하다'])]
    
    # 결과 출력
    print(f"{file_name}.csv")
    print(filter_speed_ratio)
    
    return delivery_filtered

# 함수 실행
file_path = './해시태그병합/3977488_keywords.csv'
result_df = process_delivery_data(file_path)

3977488_keywords.csv
speed
만족하다    100.00%
Name: count, dtype: object


In [561]:
# 색감 전체 함수
from konlpy.tag import Okt

def process_color_data(file_path):
    # 파일 이름 추출
    file_name = file_path.split('/')[-1].split('.')[0]
    
    # CSV 파일 읽기
    df = pd.read_csv(file_path)
    
    # '버핏' -> '오버핏' 변경
    df['common_keywords_3_plus'] = df['common_keywords_3_plus'].apply(
        lambda x: ', '.join(['오버핏' if keyword.strip() == '버핏' else keyword for keyword in str(x).split(',')]) if pd.notnull(x) else x
    )
    
    # 색감 관련 키워드 리스트
    color_keywords = ["색감", "색상", "색", "밝다", "어둡다", "밝은", "어두운", "무채색", "크림색", "흰색", "하얀색", "화이트", "빨간색", 
                      "회색", "검은색", "보랏빛", "네이비", "브라운", "베이지", "보라색", "검정", "블랙", "그레이", "블루", "초록색", "그린", 
                      "진하다", "오묘하다"]

    # '색감' 분류 (정확한 단어 매칭)
    df['color'] = df['common_keywords_3_plus'].apply(
        lambda x: '색감' if isinstance(x, str) and any(keyword.strip() in color_keywords for keyword in x.split(',')) else '-'
    )
    
    # Okt 형태소 분석기 초기화
    okt = Okt()
    
    # 'Processed_Review_Okt' 열을 만들고, 'color'가 '색감'인 행만 Okt 적용
    df['Processed_Review_Okt'] = df['Processed_Review']
    color_filter = df['color'] == '색감'
    df.loc[color_filter, 'Processed_Review_Okt'] = df.loc[color_filter, 'Processed_Review'].apply(
        lambda x: ' '.join(okt.morphs(x, stem=False)) if isinstance(x, str) else x
    )
    
    # 'Processed_Review_Okt'에서 쉼표로 연결
    df['Processed_Review_Okt'] = df['Processed_Review_Okt'].apply(
        lambda x: ', '.join(x.split()) if isinstance(x, str) else x
    )
    
    # '색감'인 행만 필터링
    color_filtered = df[df['color'] == '색감'][['Processed_Review', 'common_keywords_3_plus', 'color', 'Processed_Review_Okt']]
    
    # 밝다/어둡다 관련 키워드 리스트
    light_keywords = ["밝다", "밝은", "크림색", "흰색", "하얀색", "화이트", "빨간색", "베이지"]
    dark_keywords = ["어둡다", "어두운", "회색", "검은색", "보랏빛", "네이비", "브라운", "보라색", "검정", "블랙", "그레이", "블루", "초록색", "그린"]
    
    # brightness 열 추가하여 밝다/어둡다/만족 분류
    color_filtered['brightness'] = color_filtered['Processed_Review_Okt'].apply(
        lambda x: '생각보다 밝아요' if isinstance(x, str) and any(keyword in x for keyword in light_keywords)
        else ('생각보다 어두워요' if isinstance(x, str) and any(keyword in x for keyword in dark_keywords) else '만족해요')
    )
    
    # '생각보다 밝아요', '생각보다 어두워요', '만족해요'의 개수 계산
    color_counts = color_filtered['brightness'].value_counts()
    
    # 전체 행 수
    total_counts = len(color_filtered)
    
    # '생각보다 밝아요', '생각보다 어두워요', '만족해요'의 비율 계산
    color_ratios = (color_counts / total_counts) * 100
    color_ratios_formatted = color_ratios.apply(lambda x: f"{x:.2f}%")
    
    # 인덱스 순서를 '생각보다 밝아요', '만족해요', '생각보다 어두워요'로 맞춤
    sorted_order = ['생각보다 밝아요', '만족해요', '생각보다 어두워요']
    color_ratios_sorted = color_ratios_formatted.reindex(sorted_order)
    
    # 결과 출력
    print(f"{file_name}.csv")
    print(color_ratios_sorted)
    
    # 최종 필터된 DataFrame 반환
    return color_filtered

# 함수 실행 예시
file_path = './해시태그병합/DOODLE HEART HALF T WHITE GREYISH BLUE_keywords.csv'
result_df = process_color_data(file_path)

DOODLE HEART HALF T WHITE GREYISH BLUE_keywords.csv
brightness
생각보다 밝아요     22.35%
만족해요         70.59%
생각보다 어두워요     7.06%
Name: count, dtype: object
